<a href="https://colab.research.google.com/github/Deadkiller43/Langchain/blob/main/Task1/CLIchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from collections import deque
import torch

class ModelLoader:
    def __init__(self, model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0"):
        self.model_name = model_name

    def load(self):
        print("Loading Mistral-7B-Instruct model. This may take a few minutes on CPU...")
        tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        model = AutoModelForCausalLM.from_pretrained(
            self.model_name
        )
        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        print("Model loaded successfully.")
        return generator

class ChatMemory:
    def __init__(self, max_turns=3):
        self.memory = deque(maxlen=max_turns)

    def add(self, user_input, bot_reply):
        self.memory.append((user_input, bot_reply))

    def get_context(self):
        return "\n".join([f"User: {u}\nAssistant: {b}" for u, b in self.memory])

class Chatbot:
    def __init__(self):
        self.model_loader = ModelLoader()
        self.generator = self.model_loader.load()
        self.chat_memory = ChatMemory()

    def run(self):
        print("Chatbot started. Type /exit to quit.")
        while True:
            user_input = input("User: ")
            if user_input.strip().lower() == "/exit":
                print("Exiting chatbot. Goodbye!")
                break

            history = self.chat_memory.get_context()
            prompt = f"{history}\nUser: {user_input}\nAssistant:"

            output = self.generator(prompt, max_new_tokens=100, temperature=0.7)
            full_text = output[0]["generated_text"]
            bot_reply = full_text[len(prompt):].strip().split("\n")[0]

            print("Bot:", bot_reply)
            self.chat_memory.add(user_input, bot_reply)
if __name__ == "__main__":
    Chatbot().run()


Loading Mistral-7B-Instruct model. This may take a few minutes on CPU...
